# Kite Safari Weather Forecasting (Hurghada, Egypt)

In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt


## Load the weather data

In [ ]:
weather_data = pd.read_csv('hurghada_weather_cleaned.csv')


## Prepare datasets for forecasting

In [ ]:
# Wind Speed
weather_forecast_speed = weather_data[['date', 'wspd']].dropna().copy()
weather_forecast_speed.rename(columns={'date': 'ds', 'wspd': 'y'}, inplace=True)
weather_forecast_speed['ds'] = pd.to_datetime(weather_forecast_speed['ds'])

# Wind Direction
weather_forecast_direction = weather_data[['date', 'wdir']].dropna().copy()
weather_forecast_direction.rename(columns={'date': 'ds', 'wdir': 'y'}, inplace=True)
weather_forecast_direction['ds'] = pd.to_datetime(weather_forecast_direction['ds'])


## Train Prophet models

In [ ]:
# Wind Speed Model
model_speed = Prophet()
model_speed.fit(weather_forecast_speed)

# Wind Direction Model
model_direction = Prophet()
model_direction.fit(weather_forecast_direction)


## Forecast next 7 days

In [ ]:
# Future dataframes
future_speed = model_speed.make_future_dataframe(periods=7)
future_direction = model_direction.make_future_dataframe(periods=7)

# Predict
forecast_speed = model_speed.predict(future_speed)
forecast_direction = model_direction.predict(future_direction)

# Correct Wind Direction to 0-360 degrees
forecast_direction['yhat'] = forecast_direction['yhat'] % 360
forecast_direction['yhat_lower'] = forecast_direction['yhat_lower'] % 360
forecast_direction['yhat_upper'] = forecast_direction['yhat_upper'] % 360


## Visualize the forecasts

In [ ]:
# Wind Speed Plot
fig1 = model_speed.plot(forecast_speed)
plt.title('Wind Speed Forecast (Hurghada) [knots]', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Wind Speed (knots)')
plt.grid(True)
plt.show()

# Wind Direction Plot
fig2 = model_direction.plot(forecast_direction)
plt.title('Wind Direction Forecast (Hurghada) [degrees]', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Wind Direction (degrees)')
plt.grid(True)
plt.show()


## Display forecasted values

In [ ]:
# Last 10 forecasts for Wind Speed
print("Wind Speed Forecast (next days) [knots]:")
print(forecast_speed[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10))

# Last 10 forecasts for Wind Direction
print("\nWind Direction Forecast (next days) [degrees]:")
print(forecast_direction[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10))
